# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [ ]:
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

#Integer Encoding
label_encoder = LabelEncoder()
label_encoder.fit(df["home_ownership"])
df["home_ownership"] = label_encoder.transform(df["home_ownership"])

# Preview the data
df.dropna(axis="columns", how="all").head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)

# Create our target
target = ["loan_status"]
y = df.loc[:, target].copy()
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [7]:
# Check the balance of our target values
y["loan_status"].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
y.describe()

,loan_status
count,68817
unique,2
top,low_risk
freq,68470


In [9]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_train.shape

(51612, 92)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
data_scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [12]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [14]:
# Calculated the balanced accuracy score
y_pred_brf = brf.predict(X_test_scaled)
brf_bas = balanced_accuracy_score(y_test, y_pred_brf)
print(brf_bas)

0.8151441716926324


In [15]:
# Display the confusion matrix
c_matrix = confusion_matrix(y_test, y_pred_brf)
c_matrix_df = pd.DataFrame(c_matrix, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
c_matrix
c_matrix_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,61,26
Actual Low Risk,1213,15905


In [23]:
# Print the imbalanced classification report
display(c_matrix)
print(classification_report_imbalanced(y_test, y_pred_brf))

array([[   61,    26],
       [ 1213, 15905]], dtype=int64)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.70      0.93      0.09      0.81      0.64        87
   low_risk       1.00      0.93      0.70      0.96      0.81      0.67     17118

avg / total       0.99      0.93      0.70      0.96      0.81      0.67     17205



In [17]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sort = pd.DataFrame(sorted(zip(brf.feature_importances_, X.columns), reverse=True))
sort[:10]

,0,1
0,0.070530,total_rec_prncp
1,0.062639,last_pymnt_amnt
2,0.061965,total_rec_int
3,0.057592,total_pymnt_inv
4,0.056553,total_pymnt
5,0.027159,int_rate
6,0.019397,issue_d_Jan-2019
7,0.017538,dti
8,0.016517,installment
9,0.016505,max_bal_bc


### Easy Ensemble Classifier

In [18]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=2)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=2)

In [19]:
# Calculated the balanced accuracy score
y_pred_eec = eec.predict(X_test_scaled)
eec_bas = balanced_accuracy_score(y_test, y_pred_eec)
eec_bas

0.9331606979545628

In [20]:
# Display the confusion matrix
eec_c_matrix = confusion_matrix(y_test, y_pred_eec)
eec_c_matrix_df = pd.DataFrame(eec_c_matrix, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
eec_c_matrix
eec_c_matrix_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,80,7
Actual Low Risk,911,16207


In [22]:
# Print the imbalanced classification report
display(eec_c_matrix)
print(classification_report_imbalanced(y_test, y_pred_eec))

array([[   80,     7],
       [  911, 16207]], dtype=int64)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.92      0.95      0.15      0.93      0.87        87
   low_risk       1.00      0.95      0.92      0.97      0.93      0.87     17118

avg / total       0.99      0.95      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Balanced Random Forest Classifier had a balanced accuracy score of 0.8151441716926324.
    The Easy Ensemble Classifier had a balanced accuracy score of 0.9331606979545628.
    Therefore, the Easy Ensemble Classifier had the best (highest) balanced accuracy score.

2. Which model had the best recall score?

    The Balanced Random Forest Classifier had a recall score of (high risk: .70; low risk: 0.93; avg/total: 0.93).
    The Easy Ensemble Classifier had a recall score of (high risk: 0.92; low risk: 0.95; avg/total: 95).
    Therefore, the Easy Ensemble Classifier had the best (highest) recall score which has better scores than Balanced Random Forest Classifier in every
    category.

3. Which model had the best geometric mean score?

    The Balanced Random Forest Classifier had a geometric mean score, f1-score, of (high risk: 0.09; low risk: 0.96; avg/total: 0.96).
    The Easy Ensemble Classifier had a geometric mean score, f1-score, of (high risk: 0.15; low risk: 0.97; avg/total: 0.97).
    Therefore, the Easy Ensemble Classifier had the best (highest) geometric mean scoreas its avg/total is one one-hundreth higher than the 
    Balanced Random Forest Classifier.

4. What are the top three features?

    1: Total principal received to date (total_rec_prncp)
    2: Last total payment amount received (last_pymmt_amnt)
    3: Total interest received to date (total_rec_int)
